In [1]:
# based on: 
# https://towardsdatascience.com/deep-learning-using-transfer-learning-python-code-for-resnet50-8acdfb3a2d38

In [2]:
import PIL
from PIL import Image
import numpy as np
import pandas as pd
import os 
import random
import time 
import matplotlib.pyplot as plt
import pydot

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelBinarizer
from sklearn.metrics import classification_report, confusion_matrix, roc_curve, plot_confusion_matrix
from keras.models import Sequential
from keras.optimizers import SGD, Adam
from keras.utils import to_categorical, plot_model
from keras.callbacks import ModelCheckpoint, EarlyStopping
from keras.preprocessing.image import ImageDataGenerator
from keras.models import load_model
import tensorflow as tf
config = tf.ConfigProto()
config.gpu_options.allow_growth = True
sess = tf.Session(config=config)

from keras.layers.normalization import BatchNormalization
from keras.layers.convolutional import Conv2D
from keras.layers.convolutional import MaxPooling2D#, GlobalAveragePooling2D
from keras.layers.core import Activation
from keras.layers.core import Flatten
from keras.layers.core import Dropout
from keras.layers.core import Dense
#from keras.layers.core import GlobalAveragePooling2D
from keras import backend as K
from keras import regularizers
from keras.models import Model
import keras

from keras.applications.inception_v3 import InceptionV3
from keras.applications.inception_v3 import preprocess_input

Using TensorFlow backend.
/home/jan/anaconda3/envs/tf-gpu-clone/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:523: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/jan/anaconda3/envs/tf-gpu-clone/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:524: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/jan/anaconda3/envs/tf-gpu-clone/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:525: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/home/jan/a

In [3]:
# This block is used to load the data into RAM

# list of dirs containing the data - one for the porosity labeled data and one for the no_porosity_labeled data
dirs = ['/media/jan/Volume_big/datasets/Square_data_Gridsize_220_100µm_corrected/3layer/generated_ds/standard/porosity',
        '/media/jan/Volume_big/datasets/Square_data_Gridsize_220_100µm_corrected/3layer/generated_ds/standard/no_porosity']

length = 1869 # number of samples per category 
random_seed = 22 
image_paths = []
show_time = True  # can be toggled to show the time for loading the data into RAM 

if show_time:
    start_time = time.time()

# creating a list with paths of all files in each folder     
for d in dirs: 
    cur_paths = []
    for path in os.listdir(d):
        full_path = os.path.join(d,path)  # adding the full path as the label is stored in the path
        cur_paths.append(full_path)
    
    # randomly shuffling the created list and cutting it to the desired length
    # added to the code for being able to deal with non pre-balanced data sets - if folders both have same length nothing happens at this point
    random.seed(random_seed) 
    random.shuffle(cur_paths)
    cur_paths = cur_paths[:length] 
    
    image_paths.append(cur_paths) # appending the cut list to a new list containing all the final paths        

image_paths = [val for sublist in image_paths for val in sublist] # getting the seperate image paths out of the list
random.seed(random_seed)
random.shuffle(image_paths) # randomly shuffling the list containing the final paths

labels = []
data = []
# generating the label from the second last part of path for every element in the list and adding it to labels-list
# loading the np array, resizing and storing it to data-list

for path in image_paths:
    label = path.split(os.path.sep)[-2] # here it is 'porosity' or 'no_porosity' 
    # additionally to the pure label the whole path is stored to make a traceback of model_behaviour possible 
    # the number needs to be changed according to the link to output something like '/porosity/ZP...'
    labels.append([label,path[97:]])
    array = np.load(path)
    # img = Image.fromarray(array)   # could be added if resizing operation was necessary
    # img = img.resize((128, 128), PIL.Image.LANCZOS)
    # array_resized = np.array(img)
    data.append(array)
    
# in case of transfer learning the scaling function of the original network should be used 
data = np.array(data, dtype="float") #/ 255.0 
labels = np.array(labels)

# splitting data and labels in train, valid and test set in ratio (0.75, 0.125, 0.125)
(trainX, testX, trainY, testY) = train_test_split(data, labels, test_size=0.25, random_state=42)
(validX, testX, validY, testY) = train_test_split(testX, testY, test_size=0.5, random_state=42)

# at this point the lables are an array with entries like ['porosity', '/porosity/ZP3_Slice00690_x:1_y:1.npy']
# the following part splits up the array to generate labels and also a list with the full paths 

trainY_list = []
trainY_path = []
testY_list = []
testY_path = []
validY_list = []
validY_path = []

for row in trainY:
    trainY_list.append(row[0])
    trainY_path.append(row[1]) 
trainY = np.array(trainY_list)
trainY_path = np.array(trainY_path)

for row in testY:
    testY_list.append(row[0])
    testY_path.append(row[1]) 
testY = np.array(testY_list)
testY_path = np.array(testY_path)

for row in validY:
    validY_list.append(row[0])
    validY_path.append(row[1]) 
validY = np.array(validY_list)
validY_path = np.array(validY_path)

# the LabelBinarizer module is used to one hot encode the labels 
lb = LabelBinarizer()
trainY = lb.fit_transform(trainY)
testY = lb.transform(testY)
validY = lb.transform(validY)

# now the Labels are arrays containing either a '0' for 'no_porosity' or a '1' for 'porosity'
if show_time:
    print(time.time()-start_time)
    
# the shapes of the different data sets are printed  
print('shape training dataset: ' + str(trainX.shape))
print('shape validation dataset: ' + str(validX.shape))
print('shape test dataset: ' + str(testX.shape))

4.283123016357422
shape training dataset: (2803, 220, 220, 3)
shape validation dataset: (467, 220, 220, 3)
shape test dataset: (468, 220, 220, 3)


In [4]:
# preprocessing the inputs with the keras preprocessing function 
trainX_inception = preprocess_input(trainX)
validX_inception = preprocess_input(validX)
testX_inception = preprocess_input(testX)

In [5]:
# loading the model with the imagenet-weights but without the imagenet-classifier 
inceptionv3 = InceptionV3(include_top = False, weights = 'imagenet', input_shape = (trainX_inception[1].shape))

In [7]:
inceptionv3.layers[:2]  # just for tryout

In [10]:
# creating a dictionary with all the layer names
layer_name_dict = {}

i = 0
for layer in inceptionv3.layers:
    layer_name_dict[i] = layer
    i += 1

In [9]:
layer_name_dict

{0: <keras.engine.input_layer.InputLayer at 0x7f8b0e5bf7f0>,
 1: <keras.layers.convolutional.Conv2D at 0x7f8b0e5bfba8>,
 2: <keras.layers.normalization.BatchNormalization at 0x7f8b0e5bfac8>,
 3: <keras.layers.core.Activation at 0x7f8b34746550>,
 4: <keras.layers.convolutional.Conv2D at 0x7f8aac0ade80>,
 5: <keras.layers.normalization.BatchNormalization at 0x7f8aac1960b8>,
 6: <keras.layers.core.Activation at 0x7f8aa47c09b0>,
 7: <keras.layers.convolutional.Conv2D at 0x7f8aa453ae80>,
 8: <keras.layers.normalization.BatchNormalization at 0x7f8aa454e940>,
 9: <keras.layers.core.Activation at 0x7f8aa4577fd0>,
 10: <keras.layers.pooling.MaxPooling2D at 0x7f8aa452c978>,
 11: <keras.layers.convolutional.Conv2D at 0x7f8aa452c8d0>,
 12: <keras.layers.normalization.BatchNormalization at 0x7f8aa43bad30>,
 13: <keras.layers.core.Activation at 0x7f8aa43baf28>,
 14: <keras.layers.convolutional.Conv2D at 0x7f8aa43eda90>,
 15: <keras.layers.normalization.BatchNormalization at 0x7f8aa438cd68>,
 16: <ke

In [12]:
# via inceptionv3.summary() all the concatenate-layers were found - in order to set different concat blocks as trainable 
# concatenate bei 40, 63, 86, 100, 132, 164, 196, 228, 248, 276, 277, 307, 308

inceptionv3.layers[248:] # just for try out

In [ ]:
# 1. setting the whole network as trainable
inceptionv3.trainable = True

# 2. looping through the layer names and checking whether the layer name is in the selected range
for layer in inceptionv3.layers:
    # if layer name in range -> layer stays trainable
    if layer in inceptionv3.layers[40:]:
        layer.trainable = True
    # if layer name not in range -> layer is set to not trainable
    else:
        layer.trainable = False

# DataFrame is created to give an overview of the layers and trainablility  
layers = [(layer, layer.name, layer.trainable) for layer in inceptionv3.layers]
pd.DataFrame(layers, columns=['Layer Type', 'Layer Name', 'Layer Trainable']).tail(100)

In [ ]:
# here the final model is created from the inceptionv3 feature exxtractor and a new classifier

output = inceptionv3.layers[-1].output
output = keras.layers.Flatten()(output)
inceptionv3 = Model(inceptionv3.input, output=output)

inceptionv3.summary()

model = Sequential()
model.add(inceptionv3)
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.4))
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.4))
model.add(Dense(1, activation='sigmoid'))

In [ ]:
model.summary()

In [ ]:
# Here the number of epochs, the Batchsize, the optimizer and the Data Augmentation steps are defined 
# Finally the model is compiled 

EPOCHS = 500
BS = 32
opt = SGD(lr=0.01, momentum=0.9, decay=0.01, nesterov = True)     #'Adam' # should be the default optimizer with standard setting according to Hands-On-ML
model.compile(loss="binary_crossentropy",
              optimizer=opt,
              metrics=["accuracy"])
aug = ImageDataGenerator(vertical_flip= True,horizontal_flip=True) # data is randomly flipped horizontally as well as vertically to prevent the NN from learning local dependecies aug = ImageDataGenerator(vertical_flip= True,horizontal_flip=True) # data is randomly flipped horizontally as well as vertically to prevent the NN from learning local dependecies 

In [ ]:
# Here the actual training process is started 

# first a file is defined where the model_weights are store
filepath="model_weights/InceptionV3.best.hdf5"

# the checkpoint saved the model weights whenever the validation loss decreases 
checkpoint = ModelCheckpoint(filepath, monitor='val_loss', verbose=1, save_best_only=True, mode='min')

# EarlyStopping is set up to stop training when validation loss isn't decreasing for a number of epochs defindes by the patience parameter
es = EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=10)

# checkpoint and Early stopping are added to the callbacks list 
callbacks_list = [checkpoint,es]

# Thbe model is fit to the training data and validated with the validation data 
J = model.fit_generator(aug.flow(trainX_inception, trainY, batch_size=BS),validation_data=(validX_inception, validY), steps_per_epoch=len(trainX) // BS,epochs=EPOCHS,callbacks=callbacks_list)

In [ ]:
# This block is used for plotting the learning curve

N = np.arange(0,11)#EPOCHS)
plt.style.use("ggplot")
plt.figure
plt.plot(N, J.history["loss"], label="train_loss")
plt.plot(N, J.history["val_loss"], label="val_loss")
plt.plot(N, J.history["acc"], label="train_acc")
plt.plot(N, J.history["val_acc"], label="val_acc")
plt.title("Training Loss and Accuracy InceptionV3")
plt.xlabel("Epoch #")
plt.ylabel("Loss/Accuracy")
plt.legend()

In [ ]:
# This block is used for creating the confusion matrix and the classification report with the calculated metrics

# As a first step the models best weights are loaded before making the predicitions on the so far held back test set
model.load_weights("model_weights/InceptionV3.best.hdf5")

model_predictions = model.predict(testX_inception)
model_predictions =(model_predictions>0.5)

cm = confusion_matrix(testY, model_predictions)
cr = classification_report(testY, model_predictions)


#print(model_name)
print(cm)
print(cr)